In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
 
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
 
 
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 146442 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.24-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.24-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.24-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [ ]:
import numpy as np
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.layers.advanced_activations import LeakyReLU
from keras.datasets import cifar10
(train_features, train_labels), (test_features, test_labels) = cifar10.load_data()
model = Sequential()

model.add(Conv2D(filters=16, kernel_size=(2, 2), padding="same", activation="relu", input_shape=(train_features.shape[1:])))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

model.add(Conv2D(filters=64, kernel_size=(4, 4), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

model.add(Flatten())

model.add(Dense(25600, activation="relu"))
model.add(Dense(25600, activation="relu"))
model.add(Dense(25600, activation="relu"))
model.add(Dense(25600, activation="relu"))
model.add(Dense(10, activation="softmax"))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_features, train_labels, validation_split=0.2, epochs=10, batch_size=128, verbose=1)

In [ ]:
!ps ax | grep python

In [ ]:
!nvidia-smi

In [ ]:
!kill -9 -1

In [2]:
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys

!test -d SRGAN || git clone https://github.com/leftthomas/SRGAN
if not 'SRGAN' in sys.path:
  sys.path += ['SRGAN']
%cd SRGAN/
!pwd  

In [3]:
import sys

!test -d SRGAN_Test || git clone https://github.com/goldenbili/SRGAN_Test.git
if not 'SRGAN_Test' in sys.path:
  sys.path += ['SRGAN_Test']
%cd SRGAN_Test/
!pwd  

Cloning into 'SRGAN_Test'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 155 (delta 0), reused 0 (delta 0), pack-reused 152
Receiving objects: 100% (155/155), 84.45 KiB | 10.56 MiB/s, done.
Resolving deltas: 100% (80/80), done.
/content/SRGAN_Test
/content/SRGAN_Test


In [ ]:
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip
!unzip -q DIV2K_train_HR.zip -d data
!unzip -q DIV2K_valid_HR.zip -d data
!rm DIV2K_train_HR.zip
!rm DIV2K_valid_HR.zip

臨時操作 - 複製下載的訓練資料到 google 雲端

In [ ]:
!cp -r data/DIV2K_train_HR /content/drive/MyDrive/SRGAN/data/DIV2K_train_HR
!cp -r data/DIV2K_valid_HR /content/drive/MyDrive/SRGAN/data/DIV2K_valid_HR

常數設定

In [4]:
Path_drive_Base = "/content/drive/MyDrive/SRGAN/"
Path_data_Base = Path_drive_Base + "data/"
Path_dataInput_Base = "/content/SRGAN_Test/data/"
# Path_data_Base = "/content/SRGAN_Test/data/"
Path_snapshot = Path_drive_Base + "snapshots/"
Gan_model = 'netG_epoch_8.pth' 
Dis_model = 'netD_epoch_8.pth'

In [5]:
!mkdir data

In [6]:
Sub_Train = "01**" #@param {type:"string"}
Sub_Valid = "080*" #@param {type:"string"}
Source_Train = Path_data_Base + "DIV2K_train_HR/" + Sub_Train + ".png"
Source_Valid = Path_data_Base + "DIV2K_valid_HR/" + Sub_Valid + ".png"


Path_train = Path_dataInput_Base + "DIV2K_train_" + Sub_Train
Path_valid = Path_dataInput_Base + "DIV2K_valid_" + Sub_Valid
!mkdir $Path_train
!mkdir $Path_valid

Path_train = Path_train + "/"
!cp $Source_Train $Path_train

Path_valid = Path_valid + "/"
!cp $Source_Valid $Path_valid

# Init Date time
from datetime import datetime
now = datetime.now()
Time_Info = datetime.strftime(now,'%Y-%m-%d_%H:%M:%S') 

trainPath = Path_drive_Base + "training_results/Train_"+ Sub_Train + "_Valid_" + Sub_Valid
!mkdir $trainPath
trainPath = trainPath + '/' + datetime.strftime(now,'%Y-%m-%d_%H:%M:%S')
!mkdir $trainPath
trainPath = trainPath + '/'

statisticsPath = Path_drive_Base + "statistics/Train_"+ Sub_Train + "_Valid_" + Sub_Valid
!mkdir $statisticsPath
statisticsPath = statisticsPath + '/' + datetime.strftime(now,'%Y-%m-%d_%H:%M:%S')
!mkdir $statisticsPath
statisticsPath = statisticsPath + '/'

mkdir: cannot create directory ‘/content/drive/MyDrive/SRGAN/training_results/Train_01**_Valid_080*’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/SRGAN/statistics/Train_01**_Valid_080*’: File exists


In [ ]:
!cp ../drive/My\ Drive/SRGAN/DIV2K_train_HR.zip ./data/
!cp ../drive/My\ Drive/SRGAN/DIV2K_valid_HR.zip ./data/
!unzip -q ./data/DIV2K_train_HR.zip -d data
!unzip -q ./data/DIV2K_valid_HR.zip -d data
!rm ./data/DIV2K_train_HR.zip
!rm ./data/DIV2K_valid_HR.zip

In [ ]:
!python -c "import torch; print(torch.__version__)"

In [ ]:
!python -c "import torchvision; print(torchvision.__version__)"

In [ ]:
!pip install torchvision==0.7

In [ ]:
!pip install torch==1.6

In [ ]:
!pip install cudatoolkit=10.1

In [ ]:
! python train.py \
  --num_epochs=5 \
  --use_cuda=1 \
  --batch_size=1 \
  --snapshots_folder=$Path_snapshot \
  --snapshots_Gan=$Gan_model \
  --snapshots_Dis=$Dis_model \
  --train_path=$Path_train \
  --valid_path=$Path_valid \
  --statistics=path $statisticsPath/ \
  --epochs_path=$trainPath/ \
  --willy_test=0 \
  --do_resize=1

In [7]:
    print('$Path_snapshot:')
    print(Path_snapshot)
    print('$Gan_model')
    print(Gan_model)
    print('$Dis_model')
    print(Dis_model)
    print('$Path_train')
    print(Path_train)
    print('$Path_valid')
    print(Path_valid)
    print('$statisticsPath:')
    print(statisticsPath)
    print('$trainPath:')
    print(trainPath)

$Path_snapshot:
/content/drive/MyDrive/SRGAN/snapshots/
$Gan_model
netG_epoch_8.pth
$Dis_model
netD_epoch_8.pth
$Path_train
/content/SRGAN_Test/data/DIV2K_train_01**/
$Path_valid
/content/SRGAN_Test/data/DIV2K_valid_080*/
$statisticsPath:
/content/drive/MyDrive/SRGAN/statistics/Train_01**_Valid_080*/2021-02-23_05:10:40/
$trainPath:
/content/drive/MyDrive/SRGAN/training_results/Train_01**_Valid_080*/2021-02-23_05:10:40/


In [ ]:
! python train.py \
  --num_epochs 5 \
  --use_cuda 1 \
  --batch_size 1 \
  --snapshots_folder $Path_snapshot \
  --snapshots_Gan $Gan_model \
  --snapshots_Dis $Dis_model \
  --train_path $Path_train \
  --valid_path $Path_valid \
  --statistics_path $statisticsPath/ \
  --epochs_path $trainPath/ \
  --willy_test 0 \
  --do_resize 1

$Path_snapshot:
/content/drive/MyDrive/SRGAN/snapshots/
$Gan_model
netG_epoch_8.pth
$Dis_model
netD_epoch_8.pth
$Path_train
/content/SRGAN_Test/data/DIV2K_train_01**/
$Path_valid
/content/SRGAN_Test/data/DIV2K_valid_080*/
$statisticsPath:
/content/drive/MyDrive/SRGAN/statistics/Train_01**_Valid_080*/2021-02-23_05:10:40/
$traininPath:
/content/drive/MyDrive/SRGAN/training_results/Train_01**_Valid_080*/2021-02-23_05:10:40/
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100% 528M/528M [00:07<00:00, 74.7MB/s]
  0% 0/100 [00:00<?, ?it/s]

In [ ]:
!cp -R ./epochs/ ../drive/My\ Drive/

In [ ]:
!cp -R ./training_results ../drive/My\ Drive/

In [ ]:
!cp -R ./training_results ../drive/My\ Drive/

In [ ]:
!rm *.bmp